In [32]:
# loading in modules
import sqlite3
# from peewee import *

import pandas as pd 
from sudulunu.helpers import pp, make_num, dumper, rc
import sys 

import pickle
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

import os 
import pathlib
pathos = pathlib.Path(os.path.abspath('sent_sim.ipynb')).parent
os.chdir(pathos)
print(os.getcwd())


/Users/josh_nicholas/Repos/oz-2023-facebook-political-ads-scraper/classify_model


In [23]:
# from sklearn.datasets import fetch_20newsgroups
# categories = ['soc.religion.christian',
#               'comp.graphics']

# twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

# print(type(twenty_train))

from sklearn.datasets import make_blobs

# X, y = make_blobs(n_samples =1000, centers=2, random_state=42)

# # print(X)
# # print(y)

In [24]:

db = 'input/scraperwiki.sqlite'
con = sqlite3.connect(db)

cur = con.cursor()

# table_list = [a for a in cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")]
# print(table_list)
# [('ads_by_id',), ('ads_by_query',)]

entries = [x for x in cur.execute('SELECT * FROM ads_by_query')]
# entries = [x for x in cur.execute('SELECT * FROM ads_by_id')]

# advertisers = []
# for thingo in entries:
#     advertisers.append(thingo[2])
# print(list(set(advertisers)))

# ['From the Heart', 'Fair Australia', 'The Uluru Statement from the Heart', 'Senator Jacinta Nampijinpa Price', 
# 'Not My Voice', 'Referendum News', 'Linda Burney', 'Not Enough', 'Yes23']

print(len(entries))

print(entries[0])

con.close()

5901
('975733956964675', '104894218703531', 'the voice', 'The Singing Guitarists', "🎙️ ✨ Introducing The Singing Guitarists! ✨🎸\n\n🌟 I’m here to help women and non-binary folks learn to play and sing the guitar! 🌟\n\n🎵 My mission is to create a supportive and inclusive community where everyone can discover the joy of music. Led by me, @phillipacookman, I’m dedicated to empowering individuals traditionally underrepresented in the guitar-playing world.\n\n🎶 I’m a passionate (and professional) vocalist and guitarist and I want this platform to bridge the gender gap and provide a safe space for all singers and guitarists. Whether you're a beginner or have been making music for years, we've got you covered!\n\n🌈 I offer bespoke coaching programmes that are tailored to your needs and aspirations. From belting out your favourite Celine numbers to advanced strumming patterns and chord progressions, we cover a wide range of musical styles and genres, ensuring a well-rounded learning experience.

In [30]:
# dicto = {}
# X = []

# for thingo in entries[:2]:

#     ad_id = thingo[0]
#     advertiser_id = thingo[1]
#     name = thingo[2]
#     texto = thingo[3]
#     embeddings = model.encode([texto],convert_to_numpy=True)

#     dicto[str(embeddings[0])] = ad_id
#     X.append(embeddings)
#     # print(embeddings)

# # print(dicto)

# # sents = entries[:3]
# sents = entries.copy()
# sents = [x[3] for x in sents if x[3]]
# embeddings = model.encode(sents,convert_to_numpy=True)
# # print(embeddings)

# lenno = int(len(entries)/4)
# test = entries[:lenno]
# train = entries[lenno:]

# test = [x[3] for x in test if x[3]]
# train = [x[3] for x in train if x[3]]

# test = model.encode(test,convert_to_numpy=True)
# train = model.encode(train,convert_to_numpy=True)

train = entries
train = [x[4] for x in train if x[4]]

print(train)

print(len(train))
# train = list(set(train))
# print(len(train))
train = model.encode(train,convert_to_numpy=True)

# kmeans = KMeans(n_clusters=2, random_state=0, n_init='auto').fit(train)

# # save model
# pickle.dump(kmeans, open('models/kmeans_model.pickle', "wb"))

# load model
kmeans = pickle.load(open('models/kmeans_model.pickle', "rb"))


["🎙️ ✨ Introducing The Singing Guitarists! ✨🎸\n\n🌟 I’m here to help women and non-binary folks learn to play and sing the guitar! 🌟\n\n🎵 My mission is to create a supportive and inclusive community where everyone can discover the joy of music. Led by me, @phillipacookman, I’m dedicated to empowering individuals traditionally underrepresented in the guitar-playing world.\n\n🎶 I’m a passionate (and professional) vocalist and guitarist and I want this platform to bridge the gender gap and provide a safe space for all singers and guitarists. Whether you're a beginner or have been making music for years, we've got you covered!\n\n🌈 I offer bespoke coaching programmes that are tailored to your needs and aspirations. From belting out your favourite Celine numbers to advanced strumming patterns and chord progressions, we cover a wide range of musical styles and genres, ensuring a well-rounded learning experience.\n\n🎤 I believe singing and playing the guitar go hand in hand! Develop your vocal

In [37]:

traino = entries[0]

texto = traino[3]
embedo = model.encode([texto],convert_to_numpy=True)

# print(texto)
# print(kmeans.predict(embedo))

records = []

counter = 0
for thingo in entries:
    counter += 1
    if counter % 50 ==0:
        
        print(counter)
    old = pd.read_csv('inter/knn.csv')
    donners = old['Ad_id'].unique().tolist()
    ad_id = thingo[0]

    if ad_id in donners:
        continue
    else:
        try:
            ad_id = thingo[0]
            advertiser_id = thingo[1]
            name = thingo[2]
            texto = thingo[4]
            embedo = model.encode([texto],convert_to_numpy=True)
            prediction = kmeans.predict(embedo)

            records.append({"Ad_id": ad_id, "Advertiser": advertiser_id, "Prediction": prediction[0], "Text": texto})

            cat = pd.DataFrame.from_records(records)
            # new = pd.concat([old, cat])
            # new.drop_duplicates(subset=['Ad_id'], inplace=True)

            dumper('inter', 'knn', cat)

        except Exception as e:

            print(f"Exception is {e}")
            print(f"Line: {sys.exc_info()[-1].tb_lineno}")
            pp(cat)

pp(cat)





50
Exception is 'NoneType' object is not subscriptable
Line: 29
               Ad_id       Advertiser  Prediction  \
0    975733956964675  104894218703531           1   
1    654822746614885  160347484009125           1   
2    639357034958709  102329728050606           1   
3   1349222325683428  102329728050606           1   
4   1012946076726810  102329728050606           1   
5    848264263576265  102329728050606           1   
6    855869672539893  102329728050606           1   
7    257122653770199  102329728050606           1   
8    607309411529234  102329728050606           1   
9    820465452740415  102329728050606           1   
10   804239254697152  102329728050606           1   
11  2926122184191615  102329728050606           1   
12  6408213595913845  102329728050606           1   
13   282677851017269  102329728050606           1   
14   808626700918542  102329728050606           1   
15   304380281955874  102329728050606           1   
16   243135325337909  1023297280506

In [36]:
keen = pd.read_csv('inter/knn.csv')

keen['Count'] = 1

grp = keen.groupby(by=['Prediction'])['Count'].sum().reset_index()

print(grp)

   Prediction  Count
0           0   1752
1           1   4149
